In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#Code start here
import pandas as pd
import numpy as np
import datetime
import math
import pickle
demographics = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/demographics.csv')
credit_txn = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/cc.csv')
kplus_txn = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/kplus.csv')
train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/train.csv')
test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/test.csv')


In [0]:
credit_txn_mod = credit_txn.copy()
kplus_txn_mod = kplus_txn.copy()

credit_txn_mod['pos_dt'] = pd.to_datetime(credit_txn_mod['pos_dt'])
kplus_txn_mod['sunday'] = pd.to_datetime(kplus_txn_mod['sunday'])

credit_txn_mod['month'] = credit_txn_mod['pos_dt'].dt.month
kplus_txn_mod['month'] = kplus_txn_mod['sunday'].dt.month

credit_txn_mod.drop(['pos_dt'],axis = 1,inplace = True)
kplus_txn_mod.drop(['sunday'],axis = 1,inplace = True)

In [5]:
credit_txn_mod

,cc_no,cc_txn_amt,month
0,37069,5000,5
1,37069,12000,6
2,37069,5000,4
3,37069,1600,4
4,37069,5000,1
...,...,...,...
999108,63420,800,1
999109,63420,800,4
999110,63420,10500,2
999111,63420,1700,2


In [6]:
credit_txn_mod[credit_txn_mod['cc_no']==63420]

,cc_no,cc_txn_amt,month
999074,63420,700,2
999075,63420,600,6
999076,63420,6500,4
999077,63420,800,4
999078,63420,800,4
999079,63420,900,5
999080,63420,1100,5
999081,63420,700,4
999082,63420,800,5
999083,63420,6500,5


In [7]:
credit_txn_mod_frq = credit_txn_mod.copy()
credit_txn_mod_frq['cc_txn_count'] = credit_txn_mod_frq.groupby(['cc_no','month'])['cc_no'].transform('count')

credit_txn_mod_frq.drop(['cc_txn_amt'],axis = 1,inplace = True)
credit_txn_mod_frq.drop_duplicates(inplace=True)
credit_txn_mod_frq

,cc_no,month,cc_txn_count
0,37069,5,1
1,37069,6,1
2,37069,4,2
4,37069,1,1
5,37201,4,5
...,...,...,...
999075,63420,6,5
999076,63420,4,9
999079,63420,5,8
999085,63420,3,4


In [8]:
kplus_txn_mod

,id,kp_txn_count,kp_txn_amt,month
0,14802,2,2400,1
1,14802,9,33900,4
2,14802,7,6100,5
3,14802,3,8500,1
4,14802,2,4000,3
...,...,...,...,...
889543,16740,1,100,6
889544,8468,3,59900,5
889545,10333,1,2000,6
889546,28917,2,4100,6


In [9]:
kplus_txn_mod[kplus_txn_mod['id']==14802]

,id,kp_txn_count,kp_txn_amt,month
0,14802,2,2400,1
1,14802,9,33900,4
2,14802,7,6100,5
3,14802,3,8500,1
4,14802,2,4000,3
5,14802,12,35300,5
6,14802,1,4500,3
7,14802,6,20800,1
8,14802,11,42900,6
9,14802,11,49800,2


In [10]:
kplus_txn_mod_freq = kplus_txn_mod.copy()

kplus_txn_mod_freq['kp_txn_count_sum'] = kplus_txn_mod_freq.groupby(['id','month'])['kp_txn_count'].transform('sum')
kplus_txn_mod_freq

kplus_txn_mod_freq.drop(['kp_txn_amt','kp_txn_count'],axis = 1,inplace = True)
kplus_txn_mod_freq.drop_duplicates(inplace=True)
kplus_txn_mod_freq

,id,month,kp_txn_count_sum
0,14802,1,19
1,14802,4,19
2,14802,5,27
4,14802,3,16
8,14802,6,31
...,...,...,...
889543,16740,6,1
889544,8468,5,3
889545,10333,6,1
889546,28917,6,2


In [11]:
credit_txn_mod_frq.shape[0]

247089

In [12]:
credit_txn_mod_frq_demo = pd.merge(credit_txn_mod_frq,demographics[['id','cc_no']],how= 'left',on = 'cc_no')
credit_txn_mod_frq_demo.drop(['cc_no'],axis = 1,inplace=True)
credit_txn_mod_frq_demo.shape[0]

247089

In [13]:
credit_txn_mod_frq_demo.isna().sum()

month           0
cc_txn_count    0
id              0
dtype: int64

In [0]:
kpls_freq_matrix = pd.pivot_table(kplus_txn_mod_freq, index=['id'],columns = ['month'],values='kp_txn_count_sum', aggfunc='sum')
cc_freq_matrix = pd.pivot_table(credit_txn_mod_frq_demo, index=['id'],columns = ['month'],values='cc_txn_count', aggfunc='sum')

In [0]:
kpls_freq_matrix.reset_index(inplace = True)
kpls_freq_matrix.columns.name = None

cc_freq_matrix.reset_index(inplace = True)
cc_freq_matrix.columns.name = None


In [16]:
kpls_freq_matrix

,id,1,2,3,4,5,6
0,1,NaN,NaN,NaN,NaN,NaN,2.0
1,2,13.0,15.0,14.0,27.0,19.0,16.0
2,3,11.0,8.0,10.0,13.0,15.0,13.0
3,4,47.0,60.0,48.0,54.0,52.0,54.0
4,6,NaN,NaN,NaN,NaN,NaN,10.0
...,...,...,...,...,...,...,...
49264,64995,9.0,6.0,14.0,15.0,18.0,26.0
49265,64996,NaN,4.0,2.0,4.0,6.0,4.0
49266,64997,18.0,6.0,12.0,21.0,8.0,11.0
49267,64998,14.0,14.0,12.0,18.0,12.0,22.0


In [17]:
cc_freq_matrix

,id,1,2,3,4,5,6
0,1,1.0,2.0,NaN,NaN,1.0,1.0
1,2,5.0,7.0,7.0,13.0,2.0,8.0
2,4,NaN,NaN,NaN,13.0,12.0,11.0
3,5,2.0,1.0,NaN,NaN,NaN,NaN
4,6,NaN,NaN,NaN,NaN,NaN,3.0
...,...,...,...,...,...,...,...
51462,64993,NaN,NaN,NaN,NaN,1.0,NaN
51463,64995,10.0,19.0,13.0,5.0,2.0,1.0
51464,64997,NaN,NaN,NaN,1.0,NaN,NaN
51465,64999,2.0,1.0,NaN,8.0,7.0,NaN


In [18]:
exps_freq_matrix = pd.concat([kpls_freq_matrix,cc_freq_matrix])
#exps_matrix.reset_index(inplace = True)
exps_freq_matrix.columns.name = None
exps_freq_matrix = exps_freq_matrix.groupby(['id']).sum().reset_index()
exps_freq_matrix

,id,1,2,3,4,5,6
0,1,1.0,2.0,0.0,0.0,1.0,3.0
1,2,18.0,22.0,21.0,40.0,21.0,24.0
2,3,11.0,8.0,10.0,13.0,15.0,13.0
3,4,47.0,60.0,48.0,67.0,64.0,65.0
4,5,2.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
61159,64996,0.0,4.0,2.0,4.0,6.0,4.0
61160,64997,18.0,6.0,12.0,22.0,8.0,11.0
61161,64998,14.0,14.0,12.0,18.0,12.0,22.0
61162,64999,10.0,21.0,27.0,31.0,21.0,14.0


In [0]:
exps_freq_matrix.replace(0, np.nan, inplace=True)

In [20]:

exps_matrix_value = exps_freq_matrix.iloc[:,1:7]

exps_freq_matrix_fillavg = exps_matrix_value.apply(lambda row:  row.fillna(math.ceil(row.mean(skipna=True))), axis=1)
exps_freq_matrix_fillavg['id'] =  exps_freq_matrix['id']
exps_freq_matrix_fillavg

,1,2,3,4,5,6,id
0,1.0,2.0,2.0,2.0,1.0,3.0,1
1,18.0,22.0,21.0,40.0,21.0,24.0,2
2,11.0,8.0,10.0,13.0,15.0,13.0,3
3,47.0,60.0,48.0,67.0,64.0,65.0,4
4,2.0,1.0,2.0,2.0,2.0,2.0,5
...,...,...,...,...,...,...,...
61159,4.0,4.0,2.0,4.0,6.0,4.0,64996
61160,18.0,6.0,12.0,22.0,8.0,11.0,64997
61161,14.0,14.0,12.0,18.0,12.0,22.0,64998
61162,10.0,21.0,27.0,31.0,21.0,14.0,64999


In [21]:
exps_freq_matrix_fillavg.rename({1: 'freq_month_1', 2: 'freq_month_2', 3: 'freq_month_3', 4: 'freq_month_4', 5: 'freq_month_5', 6: 'freq_month_6'}, axis=1,inplace=True)
exps_freq_matrix_fillavg

,freq_month_1,freq_month_2,freq_month_3,freq_month_4,freq_month_5,freq_month_6,id
0,1.0,2.0,2.0,2.0,1.0,3.0,1
1,18.0,22.0,21.0,40.0,21.0,24.0,2
2,11.0,8.0,10.0,13.0,15.0,13.0,3
3,47.0,60.0,48.0,67.0,64.0,65.0,4
4,2.0,1.0,2.0,2.0,2.0,2.0,5
...,...,...,...,...,...,...,...
61159,4.0,4.0,2.0,4.0,6.0,4.0,64996
61160,18.0,6.0,12.0,22.0,8.0,11.0,64997
61161,14.0,14.0,12.0,18.0,12.0,22.0,64998
61162,10.0,21.0,27.0,31.0,21.0,14.0,64999


In [0]:
kpls_matrix = pd.pivot_table(kplus_txn_mod, index=['id'],columns = ['month'],values='kp_txn_amt', aggfunc='sum')
cc_matrix = pd.pivot_table(credit_txn_mod, index=['cc_no'],columns = ['month'],values='cc_txn_amt', aggfunc='sum')

In [23]:
kplus_txn_mod[kplus_txn_mod['id']==1]

,id,kp_txn_count,kp_txn_amt,month
822505,1,1,2000,6
822506,1,1,1500,6


In [24]:
kpls_matrix

month,1,2,3,4,5,6
id,,,,,,
1,NaN,NaN,NaN,NaN,NaN,3500.0
2,7400.0,19600.0,23500.0,63400.0,24900.0,71200.0
3,15800.0,14700.0,20200.0,49900.0,16400.0,22700.0
4,1056700.0,859000.0,715300.0,1305500.0,729000.0,1410600.0
6,NaN,NaN,NaN,NaN,NaN,13400.0
...,...,...,...,...,...,...
64995,91700.0,48400.0,144600.0,84800.0,243300.0,347700.0
64996,NaN,3800.0,3300.0,3500.0,8800.0,21800.0
64997,48300.0,1000.0,40600.0,68800.0,35000.0,7500.0


In [25]:
cc_matrix

month,1,2,3,4,5,6
cc_no,,,,,,
2,NaN,NaN,4600.0,18700.0,1600.0,2000.0
4,NaN,NaN,NaN,NaN,300000.0,240000.0
5,7000.0,5000.0,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,32000.0
7,900.0,2300.0,2200.0,1200.0,2800.0,6800.0
...,...,...,...,...,...,...
102403,11000.0,1400.0,12900.0,8400.0,9800.0,42300.0
102406,600.0,1100.0,2100.0,1100.0,600.0,3000.0
102407,1900.0,2400.0,NaN,9400.0,9800.0,NaN


Reset index and merge demographics

In [0]:
kpls_matrix.reset_index(inplace = True)
kpls_matrix.columns.name = None

cc_matrix.reset_index(inplace = True)
cc_matrix.columns.name = None


In [27]:
kpls_matrix

,id,1,2,3,4,5,6
0,1,NaN,NaN,NaN,NaN,NaN,3500.0
1,2,7400.0,19600.0,23500.0,63400.0,24900.0,71200.0
2,3,15800.0,14700.0,20200.0,49900.0,16400.0,22700.0
3,4,1056700.0,859000.0,715300.0,1305500.0,729000.0,1410600.0
4,6,NaN,NaN,NaN,NaN,NaN,13400.0
...,...,...,...,...,...,...,...
49264,64995,91700.0,48400.0,144600.0,84800.0,243300.0,347700.0
49265,64996,NaN,3800.0,3300.0,3500.0,8800.0,21800.0
49266,64997,48300.0,1000.0,40600.0,68800.0,35000.0,7500.0
49267,64998,31000.0,22000.0,15200.0,42700.0,17000.0,34100.0


In [28]:
cc_matrix

,cc_no,1,2,3,4,5,6
0,2,NaN,NaN,4600.0,18700.0,1600.0,2000.0
1,4,NaN,NaN,NaN,NaN,300000.0,240000.0
2,5,7000.0,5000.0,NaN,NaN,NaN,NaN
3,6,NaN,NaN,NaN,NaN,NaN,32000.0
4,7,900.0,2300.0,2200.0,1200.0,2800.0,6800.0
...,...,...,...,...,...,...,...
64057,102403,11000.0,1400.0,12900.0,8400.0,9800.0,42300.0
64058,102406,600.0,1100.0,2100.0,1100.0,600.0,3000.0
64059,102407,1900.0,2400.0,NaN,9400.0,9800.0,NaN
64060,102408,NaN,2000.0,4000.0,NaN,NaN,NaN


In [29]:
cc_matrix_demo = pd.merge(cc_matrix,demographics[['id','cc_no']],how = 'left',on='cc_no')
cc_matrix_demo.drop(['cc_no'],axis = 1,inplace = True)

cc_matrix_demo

,1,2,3,4,5,6,id
0,NaN,NaN,4600.0,18700.0,1600.0,2000.0,2
1,NaN,NaN,NaN,NaN,300000.0,240000.0,4
2,7000.0,5000.0,NaN,NaN,NaN,NaN,5
3,NaN,NaN,NaN,NaN,NaN,32000.0,6
4,900.0,2300.0,2200.0,1200.0,2800.0,6800.0,7
...,...,...,...,...,...,...,...
64057,11000.0,1400.0,12900.0,8400.0,9800.0,42300.0,46488
64058,600.0,1100.0,2100.0,1100.0,600.0,3000.0,40342
64059,1900.0,2400.0,NaN,9400.0,9800.0,NaN,64999
64060,NaN,2000.0,4000.0,NaN,NaN,NaN,65000


In [30]:
demographics[demographics['id'] == 1987]

,id,cc_no,gender,ocp_cd,age
4042,1987,2012,2,3.0,3
4043,1987,2585,2,3.0,3
4044,1987,33599,2,3.0,3
4045,1987,71085,2,3.0,3


In [31]:
cc_matrix[cc_matrix['cc_no']==2585]

,cc_no,1,2,3,4,5,6
1607,2585,NaN,NaN,NaN,NaN,NaN,10000.0


In [0]:

#cc_matrix_demo[cc_matrix_demo.duplicated()]


In [32]:
credit_txn[credit_txn['cc_no']==98397]

,cc_no,pos_dt,cc_txn_amt
361135,98397,2018-01-20,4700
361136,98397,2018-02-17,10000
361137,98397,2018-06-14,10000
361138,98397,2018-05-13,10000
361139,98397,2018-02-17,10000


In [33]:
cc_matrix_demo[cc_matrix_demo['id'] == 1]

,1,2,3,4,5,6,id
61565,4700.0,20000.0,NaN,NaN,10000.0,10000.0,1


In [34]:
kplus_txn[kplus_txn['id'] == 6]

,id,sunday,kp_txn_count,kp_txn_amt
818924,6,2018-06-17,1,1000
818925,6,2018-06-10,1,200
818926,6,2018-06-03,8,12200


In [35]:
kplus_txn[kplus_txn['id']==1]

,id,sunday,kp_txn_count,kp_txn_amt
822505,1,2018-06-17,1,2000
822506,1,2018-06-03,1,1500


In [36]:
exps_matrix = pd.concat([kpls_matrix,cc_matrix_demo])
#exps_matrix.reset_index(inplace = True)
exps_matrix.columns.name = None
exps_matrix = exps_matrix.groupby(['id']).sum().reset_index()
exps_matrix

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,id,1,2,3,4,5,6
0,1,4700.0,20000.0,0.0,0.0,10000.0,13500.0
1,2,14000.0,25300.0,32200.0,93800.0,26500.0,83400.0
2,3,15800.0,14700.0,20200.0,49900.0,16400.0,22700.0
3,4,1056700.0,859000.0,715300.0,1535500.0,1029000.0,1650600.0
4,5,7000.0,5000.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
61159,64996,0.0,3800.0,3300.0,3500.0,8800.0,21800.0
61160,64997,48300.0,1000.0,40600.0,70900.0,35000.0,7500.0
61161,64998,31000.0,22000.0,15200.0,42700.0,17000.0,34100.0
61162,64999,4000.0,195200.0,190700.0,79800.0,24200.0,24400.0


In [0]:
exps_matrix.replace(0, np.nan, inplace=True)

In [38]:
exps_matrix

,id,1,2,3,4,5,6
0,1,4700.0,20000.0,NaN,NaN,10000.0,13500.0
1,2,14000.0,25300.0,32200.0,93800.0,26500.0,83400.0
2,3,15800.0,14700.0,20200.0,49900.0,16400.0,22700.0
3,4,1056700.0,859000.0,715300.0,1535500.0,1029000.0,1650600.0
4,5,7000.0,5000.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
61159,64996,NaN,3800.0,3300.0,3500.0,8800.0,21800.0
61160,64997,48300.0,1000.0,40600.0,70900.0,35000.0,7500.0
61161,64998,31000.0,22000.0,15200.0,42700.0,17000.0,34100.0
61162,64999,4000.0,195200.0,190700.0,79800.0,24200.0,24400.0


In [39]:

exps_matrix_value = exps_matrix.iloc[:,1:7]

exps_matrix_fillavg = exps_matrix_value.apply(lambda row:  row.fillna(row.mean(skipna=True)), axis=1)
exps_matrix_fillavg['id'] =  exps_matrix['id']
exps_matrix_fillavg

,1,2,3,4,5,6,id
0,4700.0,20000.0,12050.0,12050.0,10000.0,13500.0,1
1,14000.0,25300.0,32200.0,93800.0,26500.0,83400.0,2
2,15800.0,14700.0,20200.0,49900.0,16400.0,22700.0,3
3,1056700.0,859000.0,715300.0,1535500.0,1029000.0,1650600.0,4
4,7000.0,5000.0,6000.0,6000.0,6000.0,6000.0,5
...,...,...,...,...,...,...,...
61159,8240.0,3800.0,3300.0,3500.0,8800.0,21800.0,64996
61160,48300.0,1000.0,40600.0,70900.0,35000.0,7500.0,64997
61161,31000.0,22000.0,15200.0,42700.0,17000.0,34100.0,64998
61162,4000.0,195200.0,190700.0,79800.0,24200.0,24400.0,64999


In [40]:
exps_matrix_fillavg.id.nunique()

61164

In [41]:
#With demographics age gender ocp_cd

exps_matrix_fillavg_demo = pd.merge(exps_matrix_fillavg,demographics[['id','gender','age','ocp_cd']],how = 'left',on='id')
exps_matrix_fillavg_demo.drop_duplicates(inplace=True)
exps_matrix_fillavg_demo

,1,2,3,4,5,6,id,gender,age,ocp_cd
0,4700.0,20000.0,12050.0,12050.0,10000.0,13500.0,1,2,5,9.0
2,14000.0,25300.0,32200.0,93800.0,26500.0,83400.0,2,2,4,3.0
4,15800.0,14700.0,20200.0,49900.0,16400.0,22700.0,3,2,3,1.0
5,1056700.0,859000.0,715300.0,1535500.0,1029000.0,1650600.0,4,2,5,3.0
7,7000.0,5000.0,6000.0,6000.0,6000.0,6000.0,5,2,4,9.0
...,...,...,...,...,...,...,...,...,...,...
97245,8240.0,3800.0,3300.0,3500.0,8800.0,21800.0,64996,2,2,3.0
97246,48300.0,1000.0,40600.0,70900.0,35000.0,7500.0,64997,2,2,3.0
97247,31000.0,22000.0,15200.0,42700.0,17000.0,34100.0,64998,1,3,9.0
97248,4000.0,195200.0,190700.0,79800.0,24200.0,24400.0,64999,2,3,3.0


In [42]:
#With demographics age gender 

exps_matrix_fillavg_demo_age_gender = pd.merge(exps_matrix_fillavg,demographics[['id','gender','age']],how = 'left',on='id')
exps_matrix_fillavg_demo_age_gender.drop_duplicates(inplace=True)
exps_matrix_fillavg_demo_age_gender

,1,2,3,4,5,6,id,gender,age
0,4700.0,20000.0,12050.0,12050.0,10000.0,13500.0,1,2,5
2,14000.0,25300.0,32200.0,93800.0,26500.0,83400.0,2,2,4
4,15800.0,14700.0,20200.0,49900.0,16400.0,22700.0,3,2,3
5,1056700.0,859000.0,715300.0,1535500.0,1029000.0,1650600.0,4,2,5
7,7000.0,5000.0,6000.0,6000.0,6000.0,6000.0,5,2,4
...,...,...,...,...,...,...,...,...,...
97245,8240.0,3800.0,3300.0,3500.0,8800.0,21800.0,64996,2,2
97246,48300.0,1000.0,40600.0,70900.0,35000.0,7500.0,64997,2,2
97247,31000.0,22000.0,15200.0,42700.0,17000.0,34100.0,64998,1,3
97248,4000.0,195200.0,190700.0,79800.0,24200.0,24400.0,64999,2,3


In [43]:
exps_matrix_fillavg_demo.isna().sum()

1          0
2          0
3          0
4          0
5          0
6          0
id         0
gender     0
age        0
ocp_cd    15
dtype: int64

In [44]:
exps_matrix_fillavg_demo.groupby(['ocp_cd']).size()

ocp_cd
1.0      1109
2.0       563
3.0     28856
4.0      2117
5.0       150
6.0       138
7.0       829
8.0        10
9.0     26570
11.0       18
12.0      196
13.0      593
dtype: int64

In [45]:
exps_matrix_fillavg_demo.ocp_cd.unique()

array([ 9.,  3.,  1.,  4.,  7., 13.,  6.,  5.,  2., 12.,  8., nan, 11.])

In [46]:
exps_matrix_fillavg_demo[exps_matrix_fillavg_demo['ocp_cd'].isna()]

,1,2,3,4,5,6,id,gender,age,ocp_cd
10116,16400.0,18000.0,19700.0,249100.0,120600.0,42200.0,5344,1,3,NaN
10777,29000.0,52400.0,140100.0,72300.0,105100.0,115000.0,5694,1,2,NaN
41900,19700.0,110000.0,88100.0,215400.0,112800.0,66500.0,23233,2,3,NaN
47263,230100.0,123300.0,207900.0,328700.0,205800.0,135700.0,26505,1,3,NaN
55771,281100.0,199300.0,1327100.0,2629300.0,1501000.0,1218900.0,31877,1,5,NaN
56850,151900.0,152900.0,85900.0,147800.0,180500.0,90700.0,32575,1,3,NaN
62185,27500.0,50200.0,30100.0,36100.0,27200.0,7100.0,36098,2,3,NaN
67152,39100.0,67300.0,134900.0,236400.0,195600.0,328300.0,39539,2,3,NaN
72493,22400.0,19700.0,26000.0,11400.0,15600.0,21200.0,43383,1,3,NaN
80511,83800.0,18000.0,6500.0,43400.0,44400.0,16800.0,49642,1,2,NaN


In [47]:
#With demographics age gender freq
train_train_some_demo = pd.merge(train,exps_matrix_fillavg_demo_age_gender,how = 'left',on = 'id')
exps_matrix_train_some_demo = pd.merge(train_train_some_demo,exps_freq_matrix_fillavg,how = 'left',on = 'id')
#exps_matrix_train
exps_matrix_train_some_demo.rename({1: 'month_1', 2: 'month_2', 3: 'month_3', 4: 'month_4', 5: 'month_5', 6: 'month_6'}, axis=1,inplace=True)
exps_matrix_train_some_demo = exps_matrix_train_some_demo[-(exps_matrix_train_some_demo['month_1'].isna())]
exps_matrix_train_some_demo = exps_matrix_train_some_demo.dropna(axis = 0, how ='any')

exps_matrix_train_some_demo.reset_index(inplace = True)
exps_matrix_train_some_demo.drop(['index'],axis = 1,inplace = True)
exps_matrix_train_some_demo

,id,income,month_1,month_2,month_3,month_4,month_5,month_6,gender,age,freq_month_1,freq_month_2,freq_month_3,freq_month_4,freq_month_5,freq_month_6
0,1,20000,4700.0,20000.0,12050.0,12050.0,10000.0,13500.0,2.0,5.0,1.0,2.0,2.0,2.0,1.0,3.0
1,2,106000,14000.0,25300.0,32200.0,93800.0,26500.0,83400.0,2.0,4.0,18.0,22.0,21.0,40.0,21.0,24.0
2,3,29000,15800.0,14700.0,20200.0,49900.0,16400.0,22700.0,2.0,3.0,11.0,8.0,10.0,13.0,15.0,13.0
3,4,61000,1056700.0,859000.0,715300.0,1535500.0,1029000.0,1650600.0,2.0,5.0,47.0,60.0,48.0,67.0,64.0,65.0
4,5,18000,7000.0,5000.0,6000.0,6000.0,6000.0,6000.0,2.0,4.0,2.0,1.0,2.0,2.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47169,49995,132000,2000.0,16400.0,12000.0,14200.0,37600.0,20500.0,1.0,6.0,2.0,4.0,3.0,4.0,6.0,9.0
47170,49996,48000,7700.0,22100.0,17700.0,24700.0,28600.0,23600.0,2.0,4.0,5.0,10.0,8.0,14.0,9.0,9.0
47171,49997,37000,124800.0,129700.0,135000.0,83700.0,146100.0,79200.0,1.0,5.0,38.0,43.0,39.0,41.0,50.0,36.0
47172,49998,19000,69400.0,20000.0,92600.0,90600.0,74100.0,56400.0,2.0,2.0,23.0,22.0,24.0,38.0,41.0,42.0


In [48]:
#With demographics age gender ocp_cd freq
train_train_all_demo = pd.merge(train,exps_matrix_fillavg_demo,how = 'left',on = 'id')
exps_matrix_train_all_demo = pd.merge(train_train_all_demo,exps_freq_matrix_fillavg,how = 'left',on = 'id')
#exps_matrix_train
exps_matrix_train_all_demo.rename({1: 'month_1', 2: 'month_2', 3: 'month_3', 4: 'month_4', 5: 'month_5', 6: 'month_6'}, axis=1,inplace=True)
exps_matrix_train_all_demo = exps_matrix_train_all_demo[-(exps_matrix_train_all_demo['month_1'].isna())]
exps_matrix_train_all_demo = exps_matrix_train_all_demo.dropna(axis = 0, how ='any')

exps_matrix_train_all_demo.reset_index(inplace = True)
exps_matrix_train_all_demo.drop(['index'],axis = 1,inplace = True)
exps_matrix_train_all_demo

,id,income,month_1,month_2,month_3,month_4,month_5,month_6,gender,age,ocp_cd,freq_month_1,freq_month_2,freq_month_3,freq_month_4,freq_month_5,freq_month_6
0,1,20000,4700.0,20000.0,12050.0,12050.0,10000.0,13500.0,2.0,5.0,9.0,1.0,2.0,2.0,2.0,1.0,3.0
1,2,106000,14000.0,25300.0,32200.0,93800.0,26500.0,83400.0,2.0,4.0,3.0,18.0,22.0,21.0,40.0,21.0,24.0
2,3,29000,15800.0,14700.0,20200.0,49900.0,16400.0,22700.0,2.0,3.0,1.0,11.0,8.0,10.0,13.0,15.0,13.0
3,4,61000,1056700.0,859000.0,715300.0,1535500.0,1029000.0,1650600.0,2.0,5.0,3.0,47.0,60.0,48.0,67.0,64.0,65.0
4,5,18000,7000.0,5000.0,6000.0,6000.0,6000.0,6000.0,2.0,4.0,9.0,2.0,1.0,2.0,2.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47159,49995,132000,2000.0,16400.0,12000.0,14200.0,37600.0,20500.0,1.0,6.0,4.0,2.0,4.0,3.0,4.0,6.0,9.0
47160,49996,48000,7700.0,22100.0,17700.0,24700.0,28600.0,23600.0,2.0,4.0,3.0,5.0,10.0,8.0,14.0,9.0,9.0
47161,49997,37000,124800.0,129700.0,135000.0,83700.0,146100.0,79200.0,1.0,5.0,9.0,38.0,43.0,39.0,41.0,50.0,36.0
47162,49998,19000,69400.0,20000.0,92600.0,90600.0,74100.0,56400.0,2.0,2.0,9.0,23.0,22.0,24.0,38.0,41.0,42.0


In [49]:
exps_matrix_train_all_demo.columns

Index(['id', 'income', 'month_1', 'month_2', 'month_3', 'month_4', 'month_5',
       'month_6', 'gender', 'age', 'ocp_cd', 'freq_month_1', 'freq_month_2',
       'freq_month_3', 'freq_month_4', 'freq_month_5', 'freq_month_6'],
      dtype='object')

In [50]:
exps_matrix_train_some_demo.columns

Index(['id', 'income', 'month_1', 'month_2', 'month_3', 'month_4', 'month_5',
       'month_6', 'gender', 'age', 'freq_month_1', 'freq_month_2',
       'freq_month_3', 'freq_month_4', 'freq_month_5', 'freq_month_6'],
      dtype='object')

In [51]:
exps_matrix_train_some_demo.isna().sum()

id              0
income          0
month_1         0
month_2         0
month_3         0
month_4         0
month_5         0
month_6         0
gender          0
age             0
freq_month_1    0
freq_month_2    0
freq_month_3    0
freq_month_4    0
freq_month_5    0
freq_month_6    0
dtype: int64

In [52]:
exps_matrix_train_all_demo.isna().sum()

id              0
income          0
month_1         0
month_2         0
month_3         0
month_4         0
month_5         0
month_6         0
gender          0
age             0
ocp_cd          0
freq_month_1    0
freq_month_2    0
freq_month_3    0
freq_month_4    0
freq_month_5    0
freq_month_6    0
dtype: int64

In [53]:
#order column exps_matrix_train_some_demo
exps_matrix_train_some_demo = exps_matrix_train_some_demo[['id','month_1','freq_month_1','month_2','freq_month_2','month_3','freq_month_3','month_4','freq_month_4','month_5','freq_month_5','month_6','freq_month_6','gender','age','income']]
exps_matrix_train_some_demo

,id,month_1,freq_month_1,month_2,freq_month_2,month_3,freq_month_3,month_4,freq_month_4,month_5,freq_month_5,month_6,freq_month_6,gender,age,income
0,1,4700.0,1.0,20000.0,2.0,12050.0,2.0,12050.0,2.0,10000.0,1.0,13500.0,3.0,2.0,5.0,20000
1,2,14000.0,18.0,25300.0,22.0,32200.0,21.0,93800.0,40.0,26500.0,21.0,83400.0,24.0,2.0,4.0,106000
2,3,15800.0,11.0,14700.0,8.0,20200.0,10.0,49900.0,13.0,16400.0,15.0,22700.0,13.0,2.0,3.0,29000
3,4,1056700.0,47.0,859000.0,60.0,715300.0,48.0,1535500.0,67.0,1029000.0,64.0,1650600.0,65.0,2.0,5.0,61000
4,5,7000.0,2.0,5000.0,1.0,6000.0,2.0,6000.0,2.0,6000.0,2.0,6000.0,2.0,2.0,4.0,18000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47169,49995,2000.0,2.0,16400.0,4.0,12000.0,3.0,14200.0,4.0,37600.0,6.0,20500.0,9.0,1.0,6.0,132000
47170,49996,7700.0,5.0,22100.0,10.0,17700.0,8.0,24700.0,14.0,28600.0,9.0,23600.0,9.0,2.0,4.0,48000
47171,49997,124800.0,38.0,129700.0,43.0,135000.0,39.0,83700.0,41.0,146100.0,50.0,79200.0,36.0,1.0,5.0,37000
47172,49998,69400.0,23.0,20000.0,22.0,92600.0,24.0,90600.0,38.0,74100.0,41.0,56400.0,42.0,2.0,2.0,19000


In [54]:
#order column exps_matrix_train_all_demo
exps_matrix_train_all_demo = exps_matrix_train_all_demo[['id','month_1','freq_month_1','month_2','freq_month_2','month_3','freq_month_3','month_4','freq_month_4','month_5','freq_month_5','month_6','freq_month_6','gender','age','ocp_cd','income']]
exps_matrix_train_all_demo 

,id,month_1,freq_month_1,month_2,freq_month_2,month_3,freq_month_3,month_4,freq_month_4,month_5,freq_month_5,month_6,freq_month_6,gender,age,ocp_cd,income
0,1,4700.0,1.0,20000.0,2.0,12050.0,2.0,12050.0,2.0,10000.0,1.0,13500.0,3.0,2.0,5.0,9.0,20000
1,2,14000.0,18.0,25300.0,22.0,32200.0,21.0,93800.0,40.0,26500.0,21.0,83400.0,24.0,2.0,4.0,3.0,106000
2,3,15800.0,11.0,14700.0,8.0,20200.0,10.0,49900.0,13.0,16400.0,15.0,22700.0,13.0,2.0,3.0,1.0,29000
3,4,1056700.0,47.0,859000.0,60.0,715300.0,48.0,1535500.0,67.0,1029000.0,64.0,1650600.0,65.0,2.0,5.0,3.0,61000
4,5,7000.0,2.0,5000.0,1.0,6000.0,2.0,6000.0,2.0,6000.0,2.0,6000.0,2.0,2.0,4.0,9.0,18000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47159,49995,2000.0,2.0,16400.0,4.0,12000.0,3.0,14200.0,4.0,37600.0,6.0,20500.0,9.0,1.0,6.0,4.0,132000
47160,49996,7700.0,5.0,22100.0,10.0,17700.0,8.0,24700.0,14.0,28600.0,9.0,23600.0,9.0,2.0,4.0,3.0,48000
47161,49997,124800.0,38.0,129700.0,43.0,135000.0,39.0,83700.0,41.0,146100.0,50.0,79200.0,36.0,1.0,5.0,9.0,37000
47162,49998,69400.0,23.0,20000.0,22.0,92600.0,24.0,90600.0,38.0,74100.0,41.0,56400.0,42.0,2.0,2.0,9.0,19000


In [0]:
#Log transformation train exps_matrix_train_some_demo
exps_matrix_train_some_demo['freq_month_1'] = np.log(exps_matrix_train_some_demo['freq_month_1'])
exps_matrix_train_some_demo['freq_month_2'] = np.log(exps_matrix_train_some_demo['freq_month_2'])
exps_matrix_train_some_demo['freq_month_3'] = np.log(exps_matrix_train_some_demo['freq_month_3'])
exps_matrix_train_some_demo['freq_month_4'] = np.log(exps_matrix_train_some_demo['freq_month_4'])
exps_matrix_train_some_demo['freq_month_5'] = np.log(exps_matrix_train_some_demo['freq_month_5'])
exps_matrix_train_some_demo['freq_month_6'] = np.log(exps_matrix_train_some_demo['freq_month_6'])

exps_matrix_train_some_demo['month_1'] = np.log(exps_matrix_train_some_demo['month_1'])
exps_matrix_train_some_demo['month_2'] = np.log(exps_matrix_train_some_demo['month_2'])
exps_matrix_train_some_demo['month_3'] = np.log(exps_matrix_train_some_demo['month_3'])
exps_matrix_train_some_demo['month_4'] = np.log(exps_matrix_train_some_demo['month_4'])
exps_matrix_train_some_demo['month_5'] = np.log(exps_matrix_train_some_demo['month_5'])
exps_matrix_train_some_demo['month_6'] = np.log(exps_matrix_train_some_demo['month_6'])

exps_matrix_train_some_demo['gender'] = np.log(exps_matrix_train_some_demo['gender'])
exps_matrix_train_some_demo['age'] = np.log(exps_matrix_train_some_demo['age'])

exps_matrix_train_some_demo['sumlog'] = exps_matrix_train_some_demo[['month_1','freq_month_1','month_2','freq_month_2','month_3','freq_month_3','month_4','freq_month_4','month_5','freq_month_5','month_6','freq_month_6','gender','age']].sum(axis=1)

exps_matrix_train_some_demo['income'] = np.log(exps_matrix_train_some_demo['income'])




In [0]:
exps_matrix_train_some_demo = exps_matrix_train_some_demo[['id','month_1','freq_month_1','month_2','freq_month_2','month_3','freq_month_3','month_4','freq_month_4','month_5','freq_month_5','month_6','freq_month_6','gender','age','sumlog','income']]


In [57]:
exps_matrix_train_some_demo

,id,month_1,freq_month_1,month_2,freq_month_2,month_3,freq_month_3,month_4,freq_month_4,month_5,freq_month_5,month_6,freq_month_6,gender,age,sumlog,income
0,1,8.455318,0.000000,9.903488,0.693147,9.396820,0.693147,9.396820,0.693147,9.210340,0.000000,9.510445,1.098612,0.693147,1.609438,61.353869,9.903488
1,2,9.546813,2.890372,10.138560,3.091042,10.379722,3.044522,11.448920,3.688879,10.184900,3.044522,11.331404,3.178054,0.693147,1.386294,84.047152,11.571194
2,3,9.667765,2.397895,9.595603,2.079442,9.913438,2.302585,10.817776,2.564949,9.705037,2.708050,10.030120,2.564949,0.693147,1.098612,76.139369,10.275051
3,4,13.870661,3.850148,13.663524,4.094345,13.480457,3.871201,14.244367,4.204693,13.844098,4.158883,14.316649,4.174387,0.693147,1.609438,110.075998,11.018629
4,5,8.853665,0.693147,8.517193,0.000000,8.699515,0.693147,8.699515,0.693147,8.699515,0.693147,8.699515,0.693147,0.693147,1.386294,57.714095,9.798127
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47169,49995,7.600902,0.693147,9.705037,1.386294,9.392662,1.098612,9.560997,1.386294,10.534759,1.791759,9.928180,2.197225,0.000000,1.791759,67.067629,11.790557
47170,49996,8.948976,1.609438,10.003333,2.302585,9.781320,2.079442,10.114559,2.639057,10.261162,2.197225,10.069002,2.197225,0.693147,1.386294,74.282763,10.778956
47171,49997,11.734468,3.637586,11.772979,3.761200,11.813030,3.663562,11.334994,3.713572,11.892047,3.912023,11.279732,3.583519,0.000000,1.609438,93.708149,10.518673
47172,49998,11.147642,3.135494,9.903488,3.091042,11.436044,3.178054,11.414209,3.637586,11.213171,3.713572,10.940224,3.737670,0.693147,0.693147,87.934492,9.852194


In [0]:
#Log transformation train exps_matrix_train_all_demo
exps_matrix_train_all_demo['freq_month_1'] = np.log(exps_matrix_train_all_demo['freq_month_1'])
exps_matrix_train_all_demo['freq_month_2'] = np.log(exps_matrix_train_all_demo['freq_month_2'])
exps_matrix_train_all_demo['freq_month_3'] = np.log(exps_matrix_train_all_demo['freq_month_3'])
exps_matrix_train_all_demo['freq_month_4'] = np.log(exps_matrix_train_all_demo['freq_month_4'])
exps_matrix_train_all_demo['freq_month_5'] = np.log(exps_matrix_train_all_demo['freq_month_5'])
exps_matrix_train_all_demo['freq_month_6'] = np.log(exps_matrix_train_all_demo['freq_month_6'])

exps_matrix_train_all_demo['month_1'] = np.log(exps_matrix_train_all_demo['month_1'])
exps_matrix_train_all_demo['month_2'] = np.log(exps_matrix_train_all_demo['month_2'])
exps_matrix_train_all_demo['month_3'] = np.log(exps_matrix_train_all_demo['month_3'])
exps_matrix_train_all_demo['month_4'] = np.log(exps_matrix_train_all_demo['month_4'])
exps_matrix_train_all_demo['month_5'] = np.log(exps_matrix_train_all_demo['month_5'])
exps_matrix_train_all_demo['month_6'] = np.log(exps_matrix_train_all_demo['month_6'])

exps_matrix_train_all_demo['gender'] = np.log(exps_matrix_train_all_demo['gender'])
exps_matrix_train_all_demo['age'] = np.log(exps_matrix_train_all_demo['age'])
exps_matrix_train_all_demo['ocp_cd'] = np.log(exps_matrix_train_all_demo['ocp_cd'])
exps_matrix_train_all_demo['sumlog'] = exps_matrix_train_all_demo[['month_1','freq_month_1','month_2','freq_month_2','month_3','freq_month_3','month_4','freq_month_4','month_5','freq_month_5','month_6','freq_month_6','gender','age','ocp_cd']].sum(axis=1)


exps_matrix_train_all_demo['income'] = np.log(exps_matrix_train_all_demo['income'])


In [0]:
exps_matrix_train_all_demo = exps_matrix_train_all_demo[['id','month_1','freq_month_1','month_2','freq_month_2','month_3','freq_month_3','month_4','freq_month_4','month_5','freq_month_5','month_6','freq_month_6','gender','age','ocp_cd','sumlog','income']]


In [60]:
exps_matrix_train_all_demo

,id,month_1,freq_month_1,month_2,freq_month_2,month_3,freq_month_3,month_4,freq_month_4,month_5,freq_month_5,month_6,freq_month_6,gender,age,ocp_cd,sumlog,income
0,1,8.455318,0.000000,9.903488,0.693147,9.396820,0.693147,9.396820,0.693147,9.210340,0.000000,9.510445,1.098612,0.693147,1.609438,2.197225,63.551094,9.903488
1,2,9.546813,2.890372,10.138560,3.091042,10.379722,3.044522,11.448920,3.688879,10.184900,3.044522,11.331404,3.178054,0.693147,1.386294,1.098612,85.145764,11.571194
2,3,9.667765,2.397895,9.595603,2.079442,9.913438,2.302585,10.817776,2.564949,9.705037,2.708050,10.030120,2.564949,0.693147,1.098612,0.000000,76.139369,10.275051
3,4,13.870661,3.850148,13.663524,4.094345,13.480457,3.871201,14.244367,4.204693,13.844098,4.158883,14.316649,4.174387,0.693147,1.609438,1.098612,111.174610,11.018629
4,5,8.853665,0.693147,8.517193,0.000000,8.699515,0.693147,8.699515,0.693147,8.699515,0.693147,8.699515,0.693147,0.693147,1.386294,2.197225,59.911320,9.798127
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47159,49995,7.600902,0.693147,9.705037,1.386294,9.392662,1.098612,9.560997,1.386294,10.534759,1.791759,9.928180,2.197225,0.000000,1.791759,1.386294,68.453924,11.790557
47160,49996,8.948976,1.609438,10.003333,2.302585,9.781320,2.079442,10.114559,2.639057,10.261162,2.197225,10.069002,2.197225,0.693147,1.386294,1.098612,75.381376,10.778956
47161,49997,11.734468,3.637586,11.772979,3.761200,11.813030,3.663562,11.334994,3.713572,11.892047,3.912023,11.279732,3.583519,0.000000,1.609438,2.197225,95.905374,10.518673
47162,49998,11.147642,3.135494,9.903488,3.091042,11.436044,3.178054,11.414209,3.637586,11.213171,3.713572,10.940224,3.737670,0.693147,0.693147,2.197225,90.131716,9.852194


In [61]:
#train set have only demo and income
train_matrix = pd.merge(train,demographics[['id','gender','ocp_cd','age']],how = 'left',on = 'id')
train_matrix.drop_duplicates(inplace=True)
train_matrix = train_matrix.dropna(axis = 0, how ='any')
train_matrix = train_matrix[['id','gender','ocp_cd','age','income']]
train_matrix['gender'] =  np.log(train_matrix['gender'])
train_matrix['ocp_cd'] =  np.log(train_matrix['ocp_cd'])
train_matrix['age'] =  np.log(train_matrix['age'])
train_matrix['income'] =  np.log(train_matrix['income'])
train_matrix['sumlog'] = train_matrix[['gender','ocp_cd','age']].sum(axis = 1)
train_matrix

,id,gender,ocp_cd,age,income,sumlog
0,1,0.693147,2.197225,1.609438,9.903488,4.499810
2,2,0.693147,1.098612,1.386294,11.571194,3.178054
4,3,0.693147,0.000000,1.098612,10.275051,1.791759
5,4,0.693147,1.098612,1.609438,11.018629,3.401197
7,5,0.693147,2.197225,1.386294,9.798127,4.276666
...,...,...,...,...,...,...
85005,49996,0.693147,1.098612,1.386294,10.778956,3.178054
85007,49997,0.000000,2.197225,1.609438,10.518673,3.806662
85009,49998,0.693147,2.197225,0.693147,9.852194,3.583519
85010,49999,0.693147,1.098612,1.098612,9.472705,2.890372


In [62]:
train_matrix = train_matrix[['id','gender','ocp_cd','age','sumlog','income']]
train_matrix

,id,gender,ocp_cd,age,sumlog,income
0,1,0.693147,2.197225,1.609438,4.499810,9.903488
2,2,0.693147,1.098612,1.386294,3.178054,11.571194
4,3,0.693147,0.000000,1.098612,1.791759,10.275051
5,4,0.693147,1.098612,1.609438,3.401197,11.018629
7,5,0.693147,2.197225,1.386294,4.276666,9.798127
...,...,...,...,...,...,...
85005,49996,0.693147,1.098612,1.386294,3.178054,10.778956
85007,49997,0.000000,2.197225,1.609438,3.806662,10.518673
85009,49998,0.693147,2.197225,0.693147,3.583519,9.852194
85010,49999,0.693147,1.098612,1.098612,2.890372,9.472705


In [63]:
train_matrix.isna().sum()

id        0
gender    0
ocp_cd    0
age       0
sumlog    0
income    0
dtype: int64

In [0]:
#For model using age ocp_cd and gender no expense
from sklearn.model_selection import train_test_split


X_no_exps = train_matrix.iloc[: ,1:5]
y_no_exps = train_matrix.iloc[:, 5:6]


X_train, X_test, y_train, y_test = train_test_split( X_no_exps, y_no_exps, test_size=0.20, random_state=42)

In [65]:
#SVR using age ocp_cd and gender no expense
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR


sc_X = StandardScaler()
sc_y = StandardScaler()
X_no = sc_X.fit_transform(X_train)
y_no = sc_y.fit_transform(y_train)

regressor_no = SVR(kernel = "rbf")
regressor_no.fit(X_no,y_no)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
    gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
    tol=0.001, verbose=False)

In [66]:
#SVR
#No expense data
test_matrix = pd.merge(test,demographics,how = 'left' ,on = 'id')
test_matrix = test_matrix.dropna(axis = 0, how ='any')
test_matrix = test_matrix[['id','gender','ocp_cd','age']]
test_matrix['gender'] =  np.log(test_matrix['gender'])
test_matrix['ocp_cd'] =  np.log(test_matrix['ocp_cd'])
test_matrix['age'] =  np.log(test_matrix['age'])
test_matrix['sumlog'] = test_matrix[['gender','ocp_cd','age']].sum(axis = 1)
test_matrix.drop_duplicates(inplace=True)
test_matrix

,id,gender,ocp_cd,age,sumlog
0,50001,0.000000,2.197225,1.098612,3.295837
1,50002,0.000000,2.197225,1.386294,3.583519
4,50003,0.000000,1.098612,1.386294,2.484907
5,50004,0.693147,1.098612,1.386294,3.178054
6,50005,0.000000,1.098612,1.386294,2.484907
...,...,...,...,...,...
17391,64996,0.693147,1.098612,0.693147,2.484907
17392,64997,0.693147,1.098612,0.693147,2.484907
17393,64998,0.000000,2.197225,1.098612,3.295837
17394,64999,0.693147,1.098612,1.098612,2.890372


In [67]:
#SVR Case 1
#No Expense data 

X = test_matrix.iloc[: ,1:5]
# First transform X-test to feature scaling
sc_X_val = sc_X.transform(X)
# Second predict the value
scaled_y_pred = regressor_no.predict(sc_X_val)
# Third - since this is scaled - we have to inverse transform
y_pred = sc_y.inverse_transform(scaled_y_pred)
actual_y_pred_no = np.exp(y_pred)
actual_y_pred_no
#y_pred

array([29045.00479536, 31203.22580088, 38121.95632824, ...,
       29045.00479536, 25814.12984671, 26021.03189971])

In [0]:
test_matrix['income'] = actual_y_pred_no

In [0]:
ans_no_exps = test_matrix[['id','income']].copy()

In [0]:
#For model some demo
from sklearn.model_selection import train_test_split


X_some_demo = exps_matrix_train_some_demo.iloc[: ,1:16]
y_some_demo = exps_matrix_train_some_demo.iloc[:, 16:17]


X_train, X_test, y_train, y_test = train_test_split( X_some_demo, y_some_demo, test_size=0.20, random_state=42)

In [71]:
#SVR some demo
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR


sc_X = StandardScaler()
sc_y = StandardScaler()
X_some = sc_X.fit_transform(X_train)
y_some = sc_y.fit_transform(y_train)

regressor_some = SVR(kernel = "rbf")
regressor_some.fit(X_some,y_some)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
    gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
    tol=0.001, verbose=False)

In [72]:
#SVR 
#SOME demo
#Log transform 
#Demographics
#freq
#SVR

test_test = pd.merge(test,exps_matrix_fillavg_demo_age_gender,how = 'left' ,on = 'id')
test_test.rename({1: 'month_1', 2: 'month_2', 3: 'month_3', 4: 'month_4', 5: 'month_5', 6: 'month_6'}, axis=1,inplace=True)
test_test = test_test.dropna(axis = 0, how ='any') 
exps_matrix_test_some_demo = pd.merge(test_test,exps_freq_matrix_fillavg,how = 'left',on = 'id')
exps_matrix_test_some_demo['month_1'] = np.log(exps_matrix_test_some_demo['month_1'])
exps_matrix_test_some_demo['month_2'] = np.log(exps_matrix_test_some_demo['month_2'])
exps_matrix_test_some_demo['month_3'] = np.log(exps_matrix_test_some_demo['month_3'])
exps_matrix_test_some_demo['month_4'] = np.log(exps_matrix_test_some_demo['month_4'])
exps_matrix_test_some_demo['month_5'] = np.log(exps_matrix_test_some_demo['month_5'])
exps_matrix_test_some_demo['month_6'] = np.log(exps_matrix_test_some_demo['month_6'])

exps_matrix_test_some_demo['freq_month_1'] = np.log(exps_matrix_test_some_demo['freq_month_1'])
exps_matrix_test_some_demo['freq_month_2'] = np.log(exps_matrix_test_some_demo['freq_month_2'])
exps_matrix_test_some_demo['freq_month_3'] = np.log(exps_matrix_test_some_demo['freq_month_3'])
exps_matrix_test_some_demo['freq_month_4'] = np.log(exps_matrix_test_some_demo['freq_month_4'])
exps_matrix_test_some_demo['freq_month_5'] = np.log(exps_matrix_test_some_demo['freq_month_5'])
exps_matrix_test_some_demo['freq_month_6'] = np.log(exps_matrix_test_some_demo['freq_month_6'])

exps_matrix_test_some_demo['gender'] = np.log(exps_matrix_test_some_demo['gender'])
exps_matrix_test_some_demo['age'] = np.log(exps_matrix_test_some_demo['age'])
exps_matrix_test_some_demo['sumlog'] = exps_matrix_test_some_demo[['month_1','freq_month_1','month_2','freq_month_2','month_3','freq_month_3','month_4','freq_month_4','month_5','freq_month_5','month_6','freq_month_6','gender','age']].sum(axis=1)

exps_matrix_test_some_demo

,id,month_1,month_2,month_3,month_4,month_5,month_6,gender,age,freq_month_1,freq_month_2,freq_month_3,freq_month_4,freq_month_5,freq_month_6,sumlog
0,50001,8.268732,10.561008,9.758462,9.918376,7.600902,9.568015,0.000000,1.098612,1.609438,2.302585,1.945910,1.945910,1.098612,1.098612,66.775175
1,50002,9.588777,8.961879,10.069002,9.560997,10.085809,9.642123,0.000000,1.386294,3.091042,2.564949,3.135494,3.218876,3.295837,3.135494,77.736574
2,50003,9.314700,9.883285,9.287301,11.235854,10.857074,10.413313,0.000000,1.386294,2.079442,2.639057,2.302585,2.890372,2.890372,2.708050,77.887699
3,50004,7.696213,8.101678,7.244228,8.630522,7.003065,8.433812,0.693147,1.386294,1.609438,1.609438,1.386294,1.609438,1.386294,1.791759,58.581620
4,50005,9.487972,10.021271,9.898475,9.758462,10.038892,11.648330,0.000000,1.386294,2.197225,1.945910,1.609438,2.564949,2.772589,3.178054,76.507861
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13985,64996,9.016756,8.242756,8.101678,8.160518,9.082507,9.989665,0.693147,0.693147,1.386294,1.386294,0.693147,1.386294,1.791759,1.386294,62.010259
13986,64997,10.785187,6.907755,10.611523,11.169026,10.463103,8.922658,0.693147,0.693147,2.890372,1.791759,2.484907,3.091042,2.079442,2.397895,74.980964
13987,64998,10.341742,9.998798,9.629051,10.661954,9.740969,10.437053,0.000000,1.098612,2.639057,2.639057,2.484907,2.890372,2.484907,3.091042,78.137521
13988,64999,8.294050,12.181780,12.158457,11.287279,10.094108,10.102338,0.693147,1.098612,2.302585,3.044522,3.295837,3.433987,3.044522,2.639057,83.670282


In [73]:
exps_matrix_test_some_demo = exps_matrix_test_some_demo[['id','month_1','freq_month_1','month_2','freq_month_2','month_3','freq_month_3','month_4','freq_month_4','month_5','freq_month_5','month_6','freq_month_6','gender','age','sumlog']]
exps_matrix_test_some_demo

,id,month_1,freq_month_1,month_2,freq_month_2,month_3,freq_month_3,month_4,freq_month_4,month_5,freq_month_5,month_6,freq_month_6,gender,age,sumlog
0,50001,8.268732,1.609438,10.561008,2.302585,9.758462,1.945910,9.918376,1.945910,7.600902,1.098612,9.568015,1.098612,0.000000,1.098612,66.775175
1,50002,9.588777,3.091042,8.961879,2.564949,10.069002,3.135494,9.560997,3.218876,10.085809,3.295837,9.642123,3.135494,0.000000,1.386294,77.736574
2,50003,9.314700,2.079442,9.883285,2.639057,9.287301,2.302585,11.235854,2.890372,10.857074,2.890372,10.413313,2.708050,0.000000,1.386294,77.887699
3,50004,7.696213,1.609438,8.101678,1.609438,7.244228,1.386294,8.630522,1.609438,7.003065,1.386294,8.433812,1.791759,0.693147,1.386294,58.581620
4,50005,9.487972,2.197225,10.021271,1.945910,9.898475,1.609438,9.758462,2.564949,10.038892,2.772589,11.648330,3.178054,0.000000,1.386294,76.507861
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13985,64996,9.016756,1.386294,8.242756,1.386294,8.101678,0.693147,8.160518,1.386294,9.082507,1.791759,9.989665,1.386294,0.693147,0.693147,62.010259
13986,64997,10.785187,2.890372,6.907755,1.791759,10.611523,2.484907,11.169026,3.091042,10.463103,2.079442,8.922658,2.397895,0.693147,0.693147,74.980964
13987,64998,10.341742,2.639057,9.998798,2.639057,9.629051,2.484907,10.661954,2.890372,9.740969,2.484907,10.437053,3.091042,0.000000,1.098612,78.137521
13988,64999,8.294050,2.302585,12.181780,3.044522,12.158457,3.295837,11.287279,3.433987,10.094108,3.044522,10.102338,2.639057,0.693147,1.098612,83.670282


In [74]:
#SVR Case 2
#Some Demo

X = exps_matrix_test_some_demo.iloc[: ,1:16]
# First transform X-test to feature scaling
sc_X_val = sc_X.transform(X)
# Second predict the value
scaled_y_pred = regressor_some.predict(sc_X_val)
# Third - since this is scaled - we have to inverse transform
y_pred = sc_y.inverse_transform(scaled_y_pred)
actual_y_pred_some = np.exp(y_pred)
actual_y_pred_some

array([24305.64097686, 25392.71689167, 32934.30542859, ...,
       27083.29188536, 28420.52858027, 21349.58820414])

In [75]:
exps_matrix_test_some_demo['income'] = actual_y_pred_some

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
ans_some_demo = exps_matrix_test_some_demo[['id','income']].copy()

In [0]:
#For model all demo
from sklearn.model_selection import train_test_split


X_all_demo = exps_matrix_train_all_demo.iloc[: ,1:17]
y_all_demo = exps_matrix_train_all_demo.iloc[:, 17:18]


X_train, X_test, y_train, y_test = train_test_split( X_all_demo, y_all_demo, test_size=0.20, random_state=42)

In [78]:
#SVR all demo
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR


sc_X = StandardScaler()
sc_y = StandardScaler()
X_all = sc_X.fit_transform(X_train)
y_all = sc_y.fit_transform(y_train)

regressor_all = SVR(kernel = "rbf")
regressor_all.fit(X_all,y_all)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
    gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
    tol=0.001, verbose=False)

In [0]:
#from sklearn.metrics import r2_score

#r2_score(actual_y, actual_y_pred)

In [80]:
test_matrix.isna().sum()

id        0
gender    0
ocp_cd    0
age       0
sumlog    0
income    0
dtype: int64

In [81]:
#SVR 
#ALL demo
#Log transform 
#Demographics
#freq
#SVR

test_test = pd.merge(test,exps_matrix_fillavg_demo,how = 'left' ,on = 'id')
test_test.rename({1: 'month_1', 2: 'month_2', 3: 'month_3', 4: 'month_4', 5: 'month_5', 6: 'month_6'}, axis=1,inplace=True)
test_test = test_test.dropna(axis = 0, how ='any') 
exps_matrix_test_all_demo = pd.merge(test_test,exps_freq_matrix_fillavg,how = 'left',on = 'id')
exps_matrix_test_all_demo['month_1'] = np.log(exps_matrix_test_all_demo['month_1'])
exps_matrix_test_all_demo['month_2'] = np.log(exps_matrix_test_all_demo['month_2'])
exps_matrix_test_all_demo['month_3'] = np.log(exps_matrix_test_all_demo['month_3'])
exps_matrix_test_all_demo['month_4'] = np.log(exps_matrix_test_all_demo['month_4'])
exps_matrix_test_all_demo['month_5'] = np.log(exps_matrix_test_all_demo['month_5'])
exps_matrix_test_all_demo['month_6'] = np.log(exps_matrix_test_all_demo['month_6'])

exps_matrix_test_all_demo['freq_month_1'] = np.log(exps_matrix_test_all_demo['freq_month_1'])
exps_matrix_test_all_demo['freq_month_2'] = np.log(exps_matrix_test_all_demo['freq_month_2'])
exps_matrix_test_all_demo['freq_month_3'] = np.log(exps_matrix_test_all_demo['freq_month_3'])
exps_matrix_test_all_demo['freq_month_4'] = np.log(exps_matrix_test_all_demo['freq_month_4'])
exps_matrix_test_all_demo['freq_month_5'] = np.log(exps_matrix_test_all_demo['freq_month_5'])
exps_matrix_test_all_demo['freq_month_6'] = np.log(exps_matrix_test_all_demo['freq_month_6'])

exps_matrix_test_all_demo['age'] = np.log(exps_matrix_test_all_demo['age'])
exps_matrix_test_all_demo['gender'] = np.log(exps_matrix_test_all_demo['gender'])
exps_matrix_test_all_demo['ocp_cd'] = np.log(exps_matrix_test_all_demo['ocp_cd'])
exps_matrix_test_all_demo['sumlog'] = exps_matrix_test_all_demo[['month_1','freq_month_1','month_2','freq_month_2','month_3','freq_month_3','month_4','freq_month_4','month_5','freq_month_5','month_6','freq_month_6','gender','age','ocp_cd']].sum(axis=1)

exps_matrix_test_all_demo

,id,month_1,month_2,month_3,month_4,month_5,month_6,gender,age,ocp_cd,freq_month_1,freq_month_2,freq_month_3,freq_month_4,freq_month_5,freq_month_6,sumlog
0,50001,8.268732,10.561008,9.758462,9.918376,7.600902,9.568015,0.000000,1.098612,2.197225,1.609438,2.302585,1.945910,1.945910,1.098612,1.098612,68.972399
1,50002,9.588777,8.961879,10.069002,9.560997,10.085809,9.642123,0.000000,1.386294,2.197225,3.091042,2.564949,3.135494,3.218876,3.295837,3.135494,79.933799
2,50003,9.314700,9.883285,9.287301,11.235854,10.857074,10.413313,0.000000,1.386294,1.098612,2.079442,2.639057,2.302585,2.890372,2.890372,2.708050,78.986311
3,50004,7.696213,8.101678,7.244228,8.630522,7.003065,8.433812,0.693147,1.386294,1.098612,1.609438,1.609438,1.386294,1.609438,1.386294,1.791759,59.680233
4,50005,9.487972,10.021271,9.898475,9.758462,10.038892,11.648330,0.000000,1.386294,1.098612,2.197225,1.945910,1.609438,2.564949,2.772589,3.178054,77.606473
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13980,64996,9.016756,8.242756,8.101678,8.160518,9.082507,9.989665,0.693147,0.693147,1.098612,1.386294,1.386294,0.693147,1.386294,1.791759,1.386294,63.108871
13981,64997,10.785187,6.907755,10.611523,11.169026,10.463103,8.922658,0.693147,0.693147,1.098612,2.890372,1.791759,2.484907,3.091042,2.079442,2.397895,76.079577
13982,64998,10.341742,9.998798,9.629051,10.661954,9.740969,10.437053,0.000000,1.098612,2.197225,2.639057,2.639057,2.484907,2.890372,2.484907,3.091042,80.334745
13983,64999,8.294050,12.181780,12.158457,11.287279,10.094108,10.102338,0.693147,1.098612,1.098612,2.302585,3.044522,3.295837,3.433987,3.044522,2.639057,84.768895


In [82]:
exps_matrix_test_some_demo.isna().sum()

id              0
month_1         0
freq_month_1    0
month_2         0
freq_month_2    0
month_3         0
freq_month_3    0
month_4         0
freq_month_4    0
month_5         0
freq_month_5    0
month_6         0
freq_month_6    0
gender          0
age             0
sumlog          0
income          0
dtype: int64

In [83]:
exps_matrix_test_all_demo = exps_matrix_test_all_demo[['id','month_1','freq_month_1','month_2','freq_month_2','month_3','freq_month_3','month_4','freq_month_4','month_5','freq_month_5','month_6','freq_month_6','gender','age','ocp_cd','sumlog']]
exps_matrix_test_all_demo

,id,month_1,freq_month_1,month_2,freq_month_2,month_3,freq_month_3,month_4,freq_month_4,month_5,freq_month_5,month_6,freq_month_6,gender,age,ocp_cd,sumlog
0,50001,8.268732,1.609438,10.561008,2.302585,9.758462,1.945910,9.918376,1.945910,7.600902,1.098612,9.568015,1.098612,0.000000,1.098612,2.197225,68.972399
1,50002,9.588777,3.091042,8.961879,2.564949,10.069002,3.135494,9.560997,3.218876,10.085809,3.295837,9.642123,3.135494,0.000000,1.386294,2.197225,79.933799
2,50003,9.314700,2.079442,9.883285,2.639057,9.287301,2.302585,11.235854,2.890372,10.857074,2.890372,10.413313,2.708050,0.000000,1.386294,1.098612,78.986311
3,50004,7.696213,1.609438,8.101678,1.609438,7.244228,1.386294,8.630522,1.609438,7.003065,1.386294,8.433812,1.791759,0.693147,1.386294,1.098612,59.680233
4,50005,9.487972,2.197225,10.021271,1.945910,9.898475,1.609438,9.758462,2.564949,10.038892,2.772589,11.648330,3.178054,0.000000,1.386294,1.098612,77.606473
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13980,64996,9.016756,1.386294,8.242756,1.386294,8.101678,0.693147,8.160518,1.386294,9.082507,1.791759,9.989665,1.386294,0.693147,0.693147,1.098612,63.108871
13981,64997,10.785187,2.890372,6.907755,1.791759,10.611523,2.484907,11.169026,3.091042,10.463103,2.079442,8.922658,2.397895,0.693147,0.693147,1.098612,76.079577
13982,64998,10.341742,2.639057,9.998798,2.639057,9.629051,2.484907,10.661954,2.890372,9.740969,2.484907,10.437053,3.091042,0.000000,1.098612,2.197225,80.334745
13983,64999,8.294050,2.302585,12.181780,3.044522,12.158457,3.295837,11.287279,3.433987,10.094108,3.044522,10.102338,2.639057,0.693147,1.098612,1.098612,84.768895


In [84]:
exps_matrix_test_all_demo

,id,month_1,freq_month_1,month_2,freq_month_2,month_3,freq_month_3,month_4,freq_month_4,month_5,freq_month_5,month_6,freq_month_6,gender,age,ocp_cd,sumlog
0,50001,8.268732,1.609438,10.561008,2.302585,9.758462,1.945910,9.918376,1.945910,7.600902,1.098612,9.568015,1.098612,0.000000,1.098612,2.197225,68.972399
1,50002,9.588777,3.091042,8.961879,2.564949,10.069002,3.135494,9.560997,3.218876,10.085809,3.295837,9.642123,3.135494,0.000000,1.386294,2.197225,79.933799
2,50003,9.314700,2.079442,9.883285,2.639057,9.287301,2.302585,11.235854,2.890372,10.857074,2.890372,10.413313,2.708050,0.000000,1.386294,1.098612,78.986311
3,50004,7.696213,1.609438,8.101678,1.609438,7.244228,1.386294,8.630522,1.609438,7.003065,1.386294,8.433812,1.791759,0.693147,1.386294,1.098612,59.680233
4,50005,9.487972,2.197225,10.021271,1.945910,9.898475,1.609438,9.758462,2.564949,10.038892,2.772589,11.648330,3.178054,0.000000,1.386294,1.098612,77.606473
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13980,64996,9.016756,1.386294,8.242756,1.386294,8.101678,0.693147,8.160518,1.386294,9.082507,1.791759,9.989665,1.386294,0.693147,0.693147,1.098612,63.108871
13981,64997,10.785187,2.890372,6.907755,1.791759,10.611523,2.484907,11.169026,3.091042,10.463103,2.079442,8.922658,2.397895,0.693147,0.693147,1.098612,76.079577
13982,64998,10.341742,2.639057,9.998798,2.639057,9.629051,2.484907,10.661954,2.890372,9.740969,2.484907,10.437053,3.091042,0.000000,1.098612,2.197225,80.334745
13983,64999,8.294050,2.302585,12.181780,3.044522,12.158457,3.295837,11.287279,3.433987,10.094108,3.044522,10.102338,2.639057,0.693147,1.098612,1.098612,84.768895


In [85]:
#SVR Case 3
#ALL Demo

X = exps_matrix_test_all_demo.iloc[: ,1:17]
# First transform X-test to feature scaling
sc_X_val = sc_X.transform(X)
# Second predict the value
scaled_y_pred = regressor_all.predict(sc_X_val)
# Third - since this is scaled - we have to inverse transform
y_pred = sc_y.inverse_transform(scaled_y_pred)
actual_y_pred_all = np.exp(y_pred)
actual_y_pred_all

array([26019.23304305, 25217.07081037, 33861.66739165, ...,
       26721.39917199, 29451.36629738, 20674.20486316])

In [86]:
exps_matrix_test_all_demo['income'] = actual_y_pred_all

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [87]:
exps_matrix_test_all_demo

,id,month_1,freq_month_1,month_2,freq_month_2,month_3,freq_month_3,month_4,freq_month_4,month_5,freq_month_5,month_6,freq_month_6,gender,age,ocp_cd,sumlog,income
0,50001,8.268732,1.609438,10.561008,2.302585,9.758462,1.945910,9.918376,1.945910,7.600902,1.098612,9.568015,1.098612,0.000000,1.098612,2.197225,68.972399,26019.233043
1,50002,9.588777,3.091042,8.961879,2.564949,10.069002,3.135494,9.560997,3.218876,10.085809,3.295837,9.642123,3.135494,0.000000,1.386294,2.197225,79.933799,25217.070810
2,50003,9.314700,2.079442,9.883285,2.639057,9.287301,2.302585,11.235854,2.890372,10.857074,2.890372,10.413313,2.708050,0.000000,1.386294,1.098612,78.986311,33861.667392
3,50004,7.696213,1.609438,8.101678,1.609438,7.244228,1.386294,8.630522,1.609438,7.003065,1.386294,8.433812,1.791759,0.693147,1.386294,1.098612,59.680233,19532.867558
4,50005,9.487972,2.197225,10.021271,1.945910,9.898475,1.609438,9.758462,2.564949,10.038892,2.772589,11.648330,3.178054,0.000000,1.386294,1.098612,77.606473,36648.192603
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13980,64996,9.016756,1.386294,8.242756,1.386294,8.101678,0.693147,8.160518,1.386294,9.082507,1.791759,9.989665,1.386294,0.693147,0.693147,1.098612,63.108871,17715.284623
13981,64997,10.785187,2.890372,6.907755,1.791759,10.611523,2.484907,11.169026,3.091042,10.463103,2.079442,8.922658,2.397895,0.693147,0.693147,1.098612,76.079577,24233.388012
13982,64998,10.341742,2.639057,9.998798,2.639057,9.629051,2.484907,10.661954,2.890372,9.740969,2.484907,10.437053,3.091042,0.000000,1.098612,2.197225,80.334745,26721.399172
13983,64999,8.294050,2.302585,12.181780,3.044522,12.158457,3.295837,11.287279,3.433987,10.094108,3.044522,10.102338,2.639057,0.693147,1.098612,1.098612,84.768895,29451.366297


In [0]:
ans_all_demo = exps_matrix_test_all_demo[['id','income']].copy()

In [89]:
temp_1 = test.copy()
temp_1 = pd.merge(temp_1,ans_all_demo,how = 'left',on='id')
temp_1

,id,income
0,50001,26019.233043
1,50002,25217.070810
2,50003,33861.667392
3,50004,19532.867558
4,50005,36648.192603
...,...,...
14995,64996,17715.284623
14996,64997,24233.388012
14997,64998,26721.399172
14998,64999,29451.366297


In [90]:
temp_1.isna().sum()

id           0
income    1015
dtype: int64

In [91]:
 ans_check_1 = temp_1[-(temp_1.income.isna())].copy()
 ans_check_1

,id,income
0,50001,26019.233043
1,50002,25217.070810
2,50003,33861.667392
3,50004,19532.867558
4,50005,36648.192603
...,...,...
14995,64996,17715.284623
14996,64997,24233.388012
14997,64998,26721.399172
14998,64999,29451.366297


In [92]:
na_test_id = temp_1[temp_1.income.isna()].copy()
na_test_id.drop(['income'],axis=1,inplace=True)
na_test_id

,id
32,50033
48,50049
67,50068
72,50073
83,50084
...,...
14925,64926
14930,64931
14942,64943
14951,64952


In [93]:
temp_2 = na_test_id.copy()
temp_2 = pd.merge(temp_2,ans_some_demo,how ='left',on='id')
temp_2

,id,income
0,50033,NaN
1,50049,NaN
2,50068,NaN
3,50073,NaN
4,50084,NaN
...,...,...
1010,64926,NaN
1011,64931,NaN
1012,64943,NaN
1013,64952,NaN


In [94]:
temp_2.isna().sum()

id           0
income    1010
dtype: int64

In [95]:
ans_check_2 = temp_2[-(temp_2.income.isna())]
ans_check_2

,id,income
37,50510,34964.034031
69,51100,17404.506704
214,53218,21822.545721
280,54336,37633.310937
787,61933,26116.050092


In [96]:
na_test_id_2 = temp_2[temp_2.income.isna()].copy()
na_test_id_2.drop(['income'],axis=1,inplace=True)
na_test_id_2


,id
0,50033
1,50049
2,50068
3,50073
4,50084
...,...
1010,64926
1011,64931
1012,64943
1013,64952


In [0]:
temp_3 = na_test_id_2.copy()
temp_3 = pd.merge(temp_3,ans_no_exps,how ='left',on='id')
ans_check_3 = temp_3.copy()

In [98]:
ans_check_3

,id,income
0,50033,26033.147310
1,50049,37655.673475
2,50068,31203.225801
3,50073,31188.185604
4,50084,37655.673475
...,...,...
1005,64926,29045.004795
1006,64931,31203.225801
1007,64943,31203.225801
1008,64952,29045.004795


In [99]:
check_file = pd.concat([ans_check_1,ans_check_2,ans_check_3])
check_file

,id,income
0,50001,26019.233043
1,50002,25217.070810
2,50003,33861.667392
3,50004,19532.867558
4,50005,36648.192603
...,...,...
1005,64926,29045.004795
1006,64931,31203.225801
1007,64943,31203.225801
1008,64952,29045.004795


In [100]:
check_file.dtypes

id          int64
income    float64
dtype: object

In [101]:
check_file

,id,income
0,50001,26019.233043
1,50002,25217.070810
2,50003,33861.667392
3,50004,19532.867558
4,50005,36648.192603
...,...,...
1005,64926,29045.004795
1006,64931,31203.225801
1007,64943,31203.225801
1008,64952,29045.004795


In [0]:
#svr_no = '/content/drive/My Drive/Colab Notebooks/M_407.sava'
#pickle.dump(regressor_no, open(svr_no, 'wb'))

In [0]:
#svr_some = '/content/drive/My Drive/Colab Notebooks/M_407.savb'
#pickle.dump(regressor_some, open(svr_some, 'wb'))

In [0]:
#svr_all = '/content/drive/My Drive/Colab Notebooks/M_407.savc'
#pickle.dump(regressor_all, open(svr_all, 'wb'))

In [0]:
from google.colab import files
check_file.to_csv('O_407.csv',index = False)
files.download('O_407.csv')